In [ ]:
# @title Iterated Growth

## IMPORTING LIBRARIES

# We'll import the typing library from Python to add type hints to our code.
from typing import Callable, List, Optional, Tuple

# Matplotlib is a useful library of visualization and plotting tools.
import matplotlib as mpl
import matplotlib.pyplot as plt

# NumPy is a popular library for mathematical functions and efficient arrays.
import numpy as np

# These libraries will allow us to make the notebook interactive.
import matplotlib.animation as animation
from IPython.display import HTML, display


## VISUALIZATION HELPERS
# Define a colour map using blue and orange for the 1D cellular automata visualization.
CMAP_1D = mpl.colors.ListedColormap(["#5790fc", "#f89c20"])


# Helper function for visualizing 1D cellular automata.
def visualize_1d(num_cells: int, num_iterations: int, rule: Callable, rule_name: str) -> None:
    """Visualizes a 1D cellular automaton simulation.

    Args:
        num_cells: The number of cells in the automaton.
        num_iterations: The number of iterations to simulate.
        rule: The rule to use for the simulation.
        rule_name: The name of the rule for display purposes.
    """
    # Use a predefined style for a clean look.
    plt.style.use("ggplot")

    image = simulate_1d_cellular_automaton(num_cells, num_iterations, rule)

    fig, ax = plt.subplots(figsize=(14, 8))

    def update(frame):
        ax.clear()
        mat = ax.matshow(image[: frame + 1], cmap=CMAP_1D)

        left, right, bottom, top = mat.get_extent()
        ax.set_xticks(np.arange(left, right + 1))
        ax.set_ylim((num_iterations - 0.5, -0.5))
        ax.set_yticks(np.arange(top, bottom + 1))
        ax.set_ylabel("← iteration", fontsize=12)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.xaxis.set_ticks_position("none")
        ax.yaxis.set_ticks_position("none")
        ax.patch.set_alpha(0)
        ax.set_title(f"1D Cellular Automaton Using {rule_name}", fontsize=16)

        return ax

    anim = animation.FuncAnimation(fig, update, frames=num_iterations, repeat=False)
    plt.close(fig)

    display(HTML(anim.to_jshtml()))


def visualize_2d(
    grid_size: int,
    num_iterations: int,
    n_neighbor: int,
    fade: bool = True,
    triangular: bool = False,
) -> None:
    """Visualizes a 2D cellular automaton simulation.

    Args:
        grid_size: The size of the grid for the cellular automaton.
        num_iterations: The number of iterations to simulate the cellular automaton for.
        n_neighbor: The number of neighbours to consider for each node (must be 4, 6, or 8).
        fade: If True, the colour of the nodes will fade over time. Defaults to True.
        triangular: If True, a triangular grid is used instead of a square one. Defaults to False.
    """
    # Use a predefined style for a clean look.
    plt.style.use("ggplot")

    grids = simulate_2d_cellular_automaton(grid_size, num_iterations, n_neighbor)

    fig, ax = plt.subplots(figsize=(14, 8))

    grid_name = f"{n_neighbor}-Neighbour {'Triangular' if triangular else 'Cartesian'}"

    def update(frame):
        ax.clear()
        grid = grids[frame]

        Y, X = np.mgrid[: grid.shape[0], : grid.shape[1]]
        X = X.flatten()
        Y = Y.flatten()
        grid = grid.flatten()

        if triangular:
            X, Y = (Y + (X - grid_size // 2) / 2.0, grid_size // 2 + 0.866 * (X - grid_size // 2))

        if fade:
            ax.scatter(X, Y, c=grid, cmap="Blues")
        else:
            ax.scatter(X, Y, c=np.sign(grid), cmap="Blues")

        ax.set_aspect("equal", "box")
        ax.invert_yaxis()
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.xaxis.set_ticks_position("none")
        ax.yaxis.set_ticks_position("none")
        ax.patch.set_alpha(0)
        ax.set_title(f"2D Cellular Automaton on a {grid_name} Grid", fontsize=16)

        return ax

    anim = animation.FuncAnimation(fig, update, frames=num_iterations, repeat=False)
    plt.close(fig)

    display(HTML(anim.to_jshtml()))


def visualize_ant_agent(grid_size: int, num_iterations: int, interval: int) -> None:
    """Visualizes a 2D cellular automaton simulation.

    Args:
        grid_size: The size of the grid for the cellular automaton.
        num_iterations: The number of iterations to simulate the cellular automaton for.
        interval: The interval at which the record grid state is recorded.
    """
    # Use a predefined style for a clean look.
    plt.style.use("ggplot")

    grids = simulate_ant_agent(grid_size, num_iterations, interval)

    fig, ax = plt.subplots(figsize=(14, 8))

    def update(frame):
        ax.clear()
        grid = grids[frame]

        Y, X = np.mgrid[: grid.shape[0], : grid.shape[1]]
        X = X.flatten()
        Y = Y.flatten()
        grid = grid.flatten()

        ax.scatter(X, Y, c=grid, cmap="binary")

        ax.set_aspect("equal", "box")
        ax.invert_yaxis()
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.xaxis.set_ticks_position("none")
        ax.yaxis.set_ticks_position("none")
        ax.patch.set_alpha(0)
        ax.set_title("Simple Ant Agent on a Gridworld", fontsize=16)
        ax.text(
            x=0.5,
            y=0.97,
            s=f"iteration = {interval * frame}",
            fontsize=12,
            ha="center",
            transform=ax.transAxes,
        )

        return ax

    anim = animation.FuncAnimation(
        fig, update, frames=num_iterations // interval + 1, repeat=False
    )
    plt.close(fig)

    display(HTML(anim.to_jshtml()))

Nature is filled with complex shapes, like the patterns on a leaf or the structure of a snowflake. But interestingly, these complex patterns are often created from simple rules. We can explore this idea using something called cellular automata. Think of cellular automata like very basic computer programs. Yet, despite being basic, they can produce surprisingly complex results! This shows us that simple rules can create intricate patterns that are hard to predict in advance. This concept is a big part of why nature is so complex and varied.

In this notebook, we'll use a programming language called Python to demonstrate some cellular automata. The code and the explanations come from Paul Charbonneau's book *Natural Complexity*; see references below.

## 1D Cellular Automata

For starters, imagine a row of cells, like squares in a line, where each cell is either blue or orange. The colour of each cell can change based on a rule that looks at the colours of the cells next to it (its neighbours). By stacking rows of these cells below each other, we can make a picture that shows how the colours change over time. Let's consider three simple but different rules:

1. A cell is colored <font color="f89c20">orange</font> if it is *already* <font color="f89c20">orange</font> or if *at least one* of its neighbors is <font color="f89c20">orange</font>; otherwise, it is colored <font color="5790fc">blue</font>.
2. A cell is colored <font color="f89c20">orange</font> if *at least one* of its neighbors is <font color="f89c20">orange</font>; otherwise, it is colored <font color="5790fc">blue</font>.
3. A cell is colored <font color="f89c20">orange</font> if *exactly one* of its neighbors is <font color="f89c20">orange</font>; otherwise, it is colored <font color="5790fc">blue</font>.

We're going to turn each of these rules into a Python function. Each function has two arguments: the entire row of the cells and then the index of the cell that we want to colour. Let's break down what those two arguments will look like. First, we'll use a list (called an array) to represent the line of cells. You can think of an array as an ordered list of values. Second, we pass the function the index of the cell to which we want to apply the rule. Each item in the array has an index: its position in the list. In most programming languages, including Python, array indices start at 0. So, the first item in the array would be at index 0, and the second element would be at index 1, and so on. When we draw our array, we'll start from left to right. For our array, we'll put a `0` to represent a blue cell and a `1` to represent an orange cell. By default, we'll start our array as all blue cells.

When we want to get the first element of an array called `array`, we write (in Python): `array[0]`. We can also let the index be a variable. So, to get the cell at position `n` in our list, we write `array[n]`. We can also perform mathematical operations on the index within those square brackets. Recall that the 0-index element is the leftmost element. So, what would be the index of the element directly to the left of the element with index `n`? `n - 1`! And the index of the right neighbour would be `n + 1`. But what if the element doesn't have a left neighbour? Would the left neighbour for `array[0]` be `array[-1]`? Python is interesting because arrays loop around, so `array[-1]` would be the rightmost element. But not all programming languages would allow that. But, to keep things simple, we'll assume that the cells on the ends of the list only have one neighbour. We'll need to account for this in our code!

Finally, let's discuss what our functions will return. We want to use these functions to modify the colour of the cells in our array. Therefore, returning the new colour for the cell we pass to the function makes sense! If we want to colour the cell blue, we return `0`. If we want to colour the cell orange, we return `1`.

Before that, let's write three helper functions that will be useful to us when we go to make our rules:
1. `get_left_neighbor`: This function tells us the cell index to the left of a given cell. If we're looking at the first cell in the list (which has an index of `0`), there's no cell to the left, so the function returns `None`.
2. `get_right_neighbor`: This function tells us the cell index to the right of a given cell. If we're looking at the last cell in the list, there's no cell to the right. If we're looking at the last cell in the list (which has an index of `len(array) - 1`, where `len(array)` is the total number of cells), there's no cell to the right, so the function returns `None`.
3. `get_cell_color`: This function tells us the colour of a cell at a given index in the list. It returns the cell's colour, which is the value stored at that index in the array. If the index is `None`, the function returns `-1`, which we can understand as an "invalid colour".

In [ ]:
def get_left_neighbor(array: np.ndarray, index: int) -> Optional[int]:
    """Returns the index of the left neighbour of a cell in a 1-dimensional array.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell whose left neighbour we want to find.

    Returns:
        The index of the left neighbour, or None if the cell is the leftmost cell in the array.
    """
    # If the index is 0, we are the leftmost cell, so we don't have a left neighbour.
    if index == 0:
        return None

    # Otherwise, our left neighbour is at index `index - 1`.
    else:
        return index - 1


def get_right_neighbor(array: np.ndarray, index: int) -> Optional[int]:
    """
    Returns the index of the right neighbour of a cell in a 1-dimensional array.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell whose right neighbour we want to find.

    Returns:
        The index of the right neighbour, or None if the cell is the rightmost cell in the array.
    """
    # `len(array)` is the length of the array.
    # `len(array) - 1` is the rightmost index because we start indexing at 0.
    # If the index is `len(array) - 1`, then we are the rightmost cell, so we don't have a right neighbour.
    if index == len(array) - 1:
        return None

    # Otherwise, our left neighbour is at index `index + 1`.
    else:
        return index + 1


def get_cell_color(array: np.ndarray, index: Optional[int]) -> int:
    """
    Returns the colour of a cell at a given index in an array.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell whose colour we want to find.

    Returns:
        The cell colour at the given index, or -1 if the index is invalid.
    """
    # If the index is `None`, the neighbour doesn't exist. So we'll return -1 to represent an invalid colour.
    if index is None:
        return -1

    # Otherwise, since `array` is an array of colours, return the value at index `index`.
    else:
        return array[index]

In [ ]:
## DEFINE THREE SIMPLE RULES

def rule_one(array: np.ndarray, index: int) -> int:
    """Implements rule one that we defined above:

    A cell is coloured orange if it is already orange or if at least one of its neighbours is orange; otherwise, it is
    coloured blue.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell we're updating.

    Returns:
        The colour of the cell in the next iteration (0 for blue, 1 for orange).
    """
    # Get the indices of the left and right neighbours of the current cell.
    left_neighbor = get_left_neighbor(array, index)
    right_neighbor = get_right_neighbor(array, index)

    # If the current cell is orange (a 1) OR either neighbour is orange, we return 1 to colour the cell orange.
    if get_cell_color(array, index) == 1 \
    or get_cell_color(array, left_neighbor) == 1 or get_cell_color(array, right_neighbor) == 1:
        return 1

    # Otherwise, we return 0 to colour the cell blue.
    else:
        return 0


def rule_two(array: np.ndarray, index: int) -> int:
    """Implements rule two that we defined above:

    A cell is coloured orange if at least one of its neighbours is orange; otherwise, it is coloured blue.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell we're updating.

    Returns:
        The colour of the cell in the next iteration (0 for blue, 1 for orange).
    """
    # Get the indices of the left and right neighbours of the current cell.
    left_neighbor = get_left_neighbor(array, index)
    right_neighbor = get_right_neighbor(array, index)

    # If either neighbour is orange (a 1), we return 1 to colour the cell orange.
    if get_cell_color(array, left_neighbor) == 1 or get_cell_color(array, right_neighbor) == 1:
        return 1

    # Otherwise, we return 0 to colour the cell blue.
    else:
        return 0


def rule_three(array: np.ndarray, index: int) -> int:
    """Implements rule three that we defined above:

    A cell is coloured orange if exactly one of its neighbours is orange; otherwise, it is coloured blue.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell we're updating.

    Returns:
        The color of the cell in the next iteration (0 for blue, 1 for orange).
    """
    # Get the indices of the left and right neighbours of the current cell.
    left_neighbor = get_left_neighbor(array, index)
    right_neighbor = get_right_neighbor(array, index)

    # If only one neighbour is orange (a 1), we return 1 to colour the cell orange.
    if get_cell_color(array, left_neighbor) == 1 and get_cell_color(array, right_neighbor) != 1 \
    or get_cell_color(array, left_neighbor) != 1 and get_cell_color(array, right_neighbor) == 1:
        return 1

    # Otherwise, we return 0 to colour the cell blue.
    else:
        return 0

Now that the rules are defined, we must code how to simulate the cellular automaton. We want to keep track of what the rows look like at each iteration step of the simulation and then stack up all the rows at the end to form an image. As such, using a 2D array is a natural solution. So instead of just a row of cells represented as a 1D array, we now have a whole grid of cells as before. This grid's width will equal the number of cells in each row. The height of this grid will be the number of iterations we will simulate.

In Python (after importing the `numpy` library), we can index a 2D array, just like we did with a 1D array. However, this time, we have two values in our index: the row and the column. Since we will use this grid to make an image, let's call our 2D array `image`. The top row of our 2D array grid can be accessed using `image[0]`. The next row would be `image[1]` and so on. What if we wanted the first column? This one is a bit tricky; you would get the leftmost column using `image[:, 0]`. In the square brackets, two values are separated by a comma. The item on the left of the comma represents the row, and the item on the right represents the column. The `0` is straightforward; you want the column at index 0. The colon is a bit of syntax that means that you want everything. So, a colon left of the comma means you want all the rows. Combining all the rows with just column 0 means you get column 0. The next column would be `image[:, 1]` and so on. This also lets us get at an individual cell as well. If I wanted the top-left corner cell, I would use `image[0, 0]`. Its right neighbour would be `image[0, 1]`: row 0, column 1. Its bottom neighbour would be `image[1, 0]`: row 1, column 0. We will use this tool in our simulation!

In the next cell, we define `simulate_1d_cellular_automaton`, a function simulating the cellular automaton for several steps. The function takes three arguments (inputs):
1. `n_cells`: This is the number of cells in the row (the width of our automaton).
2. `n_iter`: This is the number of iterations to simulate. Each iteration is a step in time where we apply the rule to update the cells.
3. `rule`: This is the rule that we apply to update the cells.

At the beginning of the function, we create a 2D array called `image` with `n_iter` rows and `n_cells` columns. This array will be used to record how the 1D cellular automaton changes over time. Each row of the array represents the state of the automaton at a particular iteration, with the topmost row (`image[0]`) being the initial state. We start by colouring all cells blue (represented by a 0). We then colour the center cell of the first row orange (represented by a 1) for the initial state of our automaton.

Then we start our simulation. For each iteration, we loop through each cell in the row and apply the provided rule to determine the colour of the cell in the current iteration based on the state of the cells in the previous iteration. We store the result at the corresponding index in the `image` array.

After the simulation is done, we return the `image` array. To run our simulation, we define some parameters and call the `simulate_1d_cellular_automaton` function with `rule_one` as the rule. We then visualize the result with the `visualize_1d` function, which we won't show here, which will give us an interactive visualization of the `image` array. We then show the same visualizations for `rule_two` and `rule_three`.

In [ ]:
## 1D 2-STATES CELLULAR AUTOMATON

def simulate_1d_cellular_automaton(n_cells: int, n_iter: int, rule: Callable[[np.ndarray, int], int]) -> np.ndarray:
    """Simulates a 1D cellular automaton based on a specified rule.

    The initial state of the automaton has one orange cell at the center, and all other cells are blue. The rule is then
    applied for `n_iter` iterations.

    Args:
        n_cells: The number of cells in the automaton.
        n_iter: The number of iterations to simulate.
        rule: The rule to use for the simulation.

    Returns:
        A 2D numpy array representing the automaton's state after `n_iter` iterations. Each row of the array represents
        the automaton's state at a particular iteration, with the topmost row being the initial state.
    """
    # Initialize the 2D array of cells of height `n_iter` and width `n_cells`, setting all cells to 0, which is blue.
    image = np.zeros((n_iter, n_cells), dtype="int")

    # To start our cellular automaton, set the center cell to orange.
    # The index of the center cell is `n_cells // 2`, where // represents the whole part of the quotient.
    # So, in our 2D array, the center cell for the first row would be in row 0 and column `n_cells // 2`.
    image[0, n_cells // 2] = 1

    # Now, we use a nested for-loop for our simulation. The outer loop will run for `n_iter - 1` simulation steps.
    # Why the -1? Because we already have our first iteration! We initialized it in the previous lines of code.
    for step in range(1, n_iter):

        # The inner loop loops through each cell in a row, from left to right
        for index in range(n_cells):

            # If we do `image[step]`, we get the row representing the current step of the iteration.
            # However, to determine the colours for this row, we need to consider the cells' state one step prior.
            # In other words, we must apply the rule to `image[step - 1]` and then update `image[step]`.
            image[step, index] = rule(image[step - 1], index)

    # We've finished simulating, so return the complete image
    return image


# Define the number of cells in the row.
# This will be the width of our image.
n_cells = 129

# Define the number of iterations we want the cellular automaton to run.
# This will be the height of our image.
n_iter = 64

# We'll use some plotting tools and the function we just defined to visualize the cellular automaton.
visualize_1d(n_cells, n_iter, rule_one, "Rule 1")

In [ ]:
# Visualize rule two.
visualize_1d(n_cells, n_iter, rule_two, "Rule 2")

In [ ]:
# Visualize rule three.
visualize_1d(n_cells, n_iter, rule_three, "Rule 3")

Using the first two rules, the patterns created are simple and predictable. However, using the third rule, the pattern becomes more complex, with white cells forming interesting shapes within the overall structure. The growth of this pattern is a mix of branching, fanning out, and extinction in the interior. These visualizations show that cellular automata can create complex and unpredictable patterns even with simple rules. Try changing some of the values in the code, such as the number of cells `n_cells`, the number of iterations `n_iter`, and how the cellular automaton is initialized (e.g., `n_cells // 2` on line 10)!

For the third rule, if we continued the simulation for an unbounded (infinite width) array, we would continue to see the pattern form blue triangles nested within each other. These triangles are made up of smaller triangles, which are also made up of even smaller triangles. This nested pattern is an example of *self-similarity*, meaning that the same pattern is found at different scales. Self-similarity is a feature of structures called *fractals*, which we will continue exploring in this notebook. The particular fractal we see with the third rule is known as a *Sierpiński triangle*.

Now, let's consider a fourth rule which isn't too much more complicated than the others:

4. Consider the current cell and its two neighbours as a group. The current cell is colored <font color="f89c20">orange</font> if *exactly one* member of the group is <font color="f89c20">orange</font> **or** if only the current cell and its right neighbor are <font color="f89c20">orange</font>; otherwise, it is colored <font color="5790fc">blue</font>.

We implement this fourth rule in the code cell below as `rule_four` and visualize it as we did with the others:

In [ ]:
def rule_four(array: np.ndarray, index: int) -> int:
    """Implements rule four that we defined above:

    Consider the current cell and its two neighbours as a group. The current cell is coloured orange if exactly one
    group member is orange or if only the current cell and its right neighbour are orange; otherwise, it is coloured
    blue.

    Args:
        array: A 1-dimensional array of cell colours.
        index: The index of the cell we're updating.

    Returns:
        The colour of the cell in the next iteration (0 for blue, 1 for orange).
    """
    # Get the indices of the left and right neighbours of the current cell.
    left_neighbor = get_left_neighbor(array, index)
    right_neighbor = get_right_neighbor(array, index)

    # If exactly one of the cell's left neighbour, itself, and its right neighbour are orange (a 1), we return 1 to colour it orange.
    if get_cell_color(array, left_neighbor) == 1 and get_cell_color(array, index) != 1 and get_cell_color(array, right_neighbor) != 1 \
    or get_cell_color(array, left_neighbor) != 1 and get_cell_color(array, index) == 1 and get_cell_color(array, right_neighbor) != 1 \
    or get_cell_color(array, left_neighbor) != 1 and get_cell_color(array, index) != 1 and get_cell_color(array, right_neighbor) == 1:
        return 1

    # Or, if only the cell and its right neighbour are orange, also return 1 to colour the cell orange.
    elif get_cell_color(array, left_neighbor) != 1 and get_cell_color(array, index) == 1 and get_cell_color(array, right_neighbor) == 1:
        return 1

    # Otherwise, we return 0 to colour the cell blue.
    else:
        return 0


# Create a plot just for rule four.
visualize_1d(n_cells, n_iter, rule_four, "Rule 4")

The pattern created has a directional bias, meaning it's not symmetrical. The pattern forms a triangle, but the left side looks more regular and organized, while the right side seems random.

There are 256 ($2^8$) possible cellular automata rules based on three neighbouring cells and two states. As we've seen, these rules can create various patterns, from simple triangles and checkerboards to more complex and random designs. We group these rules into four classes based on the patterns they create:

1. Class I cellular automata exhibit patterns that don't change much over time.
2. Class II cellular automata exhibit patterns where the colours cycle predictably.
3. Class III cellular automata exhibit patterns that don't follow a predictable cycle.
4. Class IV cellular automata exhibit more complex and interesting patterns that don't fit into the other three classes.

Each rule we've looked at produces a pattern that belongs to its respective class (i.e., rule 1 produces a Class I cellular automata, rule 2 produces a Class II cellular automata, and so on).

## 2D Cellular Automata

Cellular automata are systems of individual cells that change based on rules. These systems can be expanded to two or more dimensions, which adds complexity as you have to specify how the cells interact (their "connectivity"). It's handy to think of these systems as networks of nodes, where each node is like the center of a cell.

For example, suppose you're working with a two-dimensional grid (like a chessboard). In that case, a node usually interacts with its four nearest neighbours: left, right, top, and bottom. Sometimes, it might interact with the four diagonal neighbours too. You can also create different connectivities by modifying the grid. For example, by shifting the rows slightly and squishing the columns, you can create a triangular grid where each node has six neighbours.

In the context of cellular automata, different grid shapes with the same connectivities work the same way. All these grids can be stored in a computer's memory as two-dimensional arrays, and how we define the neighbours' connectivity dictates the grid's actual shape.

Let's consider a simple 2D cellular automata rule:

- A node becomes <font color="5790fc">active</font> if one and only one of its neighbour's nodes is also <font color="5790fc">active</font>.

This rule doesn't care about which direction the active neighbour is in, and once a node gets activated, it stays that way.

In the Python code below, we'll implement this rule and visualize what patterns emerge when applying this rule to various 2D grids. For starters, we need to determine the neighbours to a node again. Each node is indexed in the grid using two coordinates: `x` and `y`. In a 1D world, a cell would only be indexed by a single coordinate, `x`; its left neighbour was `x - 1`, and its right neighbour was `x + 1`. Similarly, in this 2D world, the left neighbour would be `(x - 1, y)`, and the right neighbour would be `(x + 1, y)`. Now, we also have the top and bottom neighbours, which have the coordinates `(x, y - 1)` and `(x, y + 1)`, respectively. It should be noted that we're using the convention that the `x`-axis increases from left to right, and the `y`-axis increases from top to bottom. Diagonal neighbours would be indexed similarly.

The code below takes the `x` and `y` as input and returns a list of the neighbours to that node. It also takes a `n_neighbors` argument which controls how many neighbours to return. For simplicity, we only accept 4, 6, or 8 neighbours. Of course, you could define the neighbours in any way you'd like. When considering four neighbours, we look at the orthogonal neighbours: right, left, top, and bottom. We add two opposite diagonal neighbours when considering six neighbours: top-right and bottom-left. And when we consider eight neighbours, we add the two remaining diagonal neighbours to the list.

We also should account for edge cases (literally!) in our code. A node on the grid's edge will have fewer neighbours than others. Node coordinates should never be negative and should never be greater than the size of the grid. We pass the width and height of the grid as additional parameters to the function `x_max` and `y_max`. These values will be the same in our example since we use a square or rhombic grid. However, separating these two values is good since we may want to look at rectangular grids in the future.

In [ ]:
def get_2d_neighbors(n_neighbors: int, x: int, y: int, x_max: int, y_max: int) -> List[Tuple[int, int]]:
    """Calculate and return the valid neighbours of a node at position (x, y).

    Args:
        n_neighbors: The number of neighbours to consider (must be 4, 6, or 8).
        x: The x-coordinate of the node.
        y: The y-coordinate of the node.
        x_max: The maximum x-coordinate value of the grid.
        y_max: The maximum y-coordinate value of the grid.

    Returns:
        A list of the (x, y) coordinates for each neighbour.
    """
    # We'll only consider 4, 6, or 8 neighbours. Anything else, and we return an empty list.
    if n_neighbors not in (4, 6, 8):
        return []

    # Otherwise, start with the orthogonal neighbours:
    neighbors = [
        (x + 1, y),  # right neighbor
        (x, y - 1),  # top neighbor
        (x - 1, y),  # left neighbor
        (x, y + 1),  # bottom neighbor
    ]

    if n_neighbors == 6:
        # Add two diagonal neighbours to the list:
        neighbors += [
            (x + 1, y - 1),  # top-right neighbour
            (x - 1, y + 1),  # bottom-left neighbour
        ]

    # Add all four diagonal neighbours to the list:
    if n_neighbors == 8:
        neighbors += [
            (x + 1, y - 1),  # top-right neighbour
            (x - 1, y - 1),  # top-left neighbour
            (x - 1, y + 1),  # bottom-left neighbour
            (x + 1, y + 1),  # bottom-right neighbour
        ]

    # Remove any invalid neighbours, such as negative indices.
    neighbors = [(i, j) for (i, j) in neighbors if 0 <= i < x_max and 0 <= j < y_max]

    # Return the list of neighbours.
    return neighbors

Next, we'll define a function to simulate a two-dimensional (2D) cellular automaton. The function `simulate_2d_cellular_automaton` takes three arguments:
1. `grid_size`: The size of the grid on which the cellular automaton operates. The width and height of this grid will be the same.
2. `n_iter`: The number of iterations to simulate the cellular automaton.
3. `n_neighbor`: The number of neighbouring cells to consider for each cell in the grid. This value must be either 4, 6, or 8 neighbours in our code.

When we simulated a 1D cellular automaton, we used a 2D array to store it. This is because we used the first axis of the array as a time axis. So `image[0]` referred to the cellular automaton's initial state. Similarly, we'll use a 3D array to store our 2D cellular automaton. Let's call this array `grid`. `grid[0]` would be the grid's initial state, `grid[1]` would be the state after 1 iteration, and so on. The function will return this 3D array `grid`, which will then be used to visualize the cellular automaton.

The simulation begins with the center node of the grid in the initial state being activated. Then, the function enters a loop over the number of iterations. For each iteration, it first copies the state of the grid from the previous iteration. Then, it loops over each cell in the grid and applies the abovementioned activation rule. At the end of the simulation, the function returns the 3D array `grid`.

Once this simulation function is defined, we will use a custom plotting function to visualize `grid`. Again, this function's details are unimportant, so focus on the visualizations. Below, we'll use a 64x64 grid and simulate it for 30 iterations. We'll first look at an 8-neighbour Cartesian grid, followed by a 6-neighbour triangular grid.

In [ ]:
## 2D 2-STATES CELLULAR AUTOMATON

def simulate_2d_cellular_automaton(grid_size: int, n_iter: int, n_neighbor: int) -> np.ndarray:
    """Simulates a 2D cellular automaton based on the following rule:

    A node becomes active if one and only one of its neighbour nodes is also active.

    Args:
        grid_size: The size of the grid for the cellular automaton.
        n_iter: The number of iterations to simulate the cellular automaton.
        n_neighbor: The number of neighbours to consider for each node (must be either 4, 6, or 8).

    Returns:
        A 3D array representing the states of the cellular automaton over time. The first dimension is time and the
        second and third are the rows and columns of the grid.
    """
    # Initialize a 3D array for our grid. Think of this as a list of 2D grids.
    # Each grid of nodes is of size `grid_size` x `grid_size`.
    grid = np.zeros((n_iter, grid_size, grid_size), dtype="int")

    # To start our cellular automaton, activate the center node, setting it to 1.
    # Note that the first index value is 0 because we're setting the state for the first step.
    grid[0, grid_size // 2, grid_size // 2] = 1

    # Now, we simulate the 2D cellular automaton for `n_iter` steps.
    for step in range(1, n_iter):

        # First, copy over the previous grid state to the current state.
        grid[step] = grid[step - 1]

        # Here, we use a nested for-loop (a for-loop inside a for-loop) to look at each individual node.
        for row in range(grid_size):
            for col in range(grid_size):

                # If a node is not activated, consider its neighbours. If any are activated, then we need to activate
                # this node. Note that the first index value corresponds to our iteration step.
                if grid[step, row, col] == 0:

                    # Use the helper function we defined to get the indices of each neighbour node
                    neighbors = get_2d_neighbors(n_neighbor, row, col, grid_size, grid_size)

                    # Get the values of the neighbour nodes. If the node value is nonzero, that means that the node is
                    # activated.
                    neighbor_values = [grid[step - 1, x, y] for x, y in neighbors]

                    # If one and only one of the neighbour nodes are activated, then we activate this node.
                    # We set the node value as the current step plus one to keep track of the order in which the nodes
                    # are activated.
                    if np.count_nonzero(neighbor_values) == 1:
                        grid[step, row, col] = step + 1

    return grid


# Define the size of the grid.
grid_size = 64

# Define the number of iterations we want the cellular automaton to run.
n_iter = 30

# Define the number of neighbour nodes to each node.
n_neighbor = 8

# Use a custom plotting function to visualize the cellular automaton.
# We specify that we want the colour of the nodes to fade over time and to use an 8-neighbour Cartesian grid.
visualize_2d(grid_size, n_iter, n_neighbor, fade=True, triangular=False)

We start with one active point in the middle. After one iteration, we get a 3x3 square of active nodes. But in the next iteration, new nodes can only activate along the diagonal lines. After this, the growth keeps happening from the corners, generating new squares, and so on. Note that the growth is self-similar.

In [ ]:
# Now, visualize the same rule on a 6-neighbour triangular grid.
n_neighbor = 6
visualize_2d(grid_size, n_iter, n_neighbor, fade=True, triangular=True)

Here, we again start with one active node. In the next iteration, we get a ring of six new active nodes around the original node. In the next iteration, the ring grows six "arms", similar to what we saw with the previous grid. As these arms grow, they fill the space between them, creating a hexagon shape. After this, growth can only happen at the corners and then towards the centers, adding another "layer" to the structure. This structure also exhibits self-similarity. You might also be thinking this looks like a snowflake! Amazing how a simple computer simulation with triangles can create something that looks like a natural snowflake.

Our conclusions after observing 2D cellular automata are similar to what we found with 1D cellular automata: even straightforward growth rules can create big structures that can be very regular or very complex. And the surprising part is that we couldn't predict what most of these structures would look like!

# Agents, ants, and highways

In the cellular automata we've looked at, the active parts are the cells or nodes on the grid, which stay put. But there's another way to think about growing patterns that involve moving parts, which we'll call "Agents". Imagine such an agent as an ant in a 2D grid world. We'll use a square (Cartesian) grid for simplicity. These ants can move around on the grid and interact with it and each other based on specific rules.

For our example, the ant moves around on the grid every iteration following these three rules:
1. Move one node forward.
2. If on a white node, colour that node <font color="black">black</font> and turn right.
3. If on a black node, colour that node <font color="white">white</font> and turn left.

Even though these rules are pretty simple, they can yield surprising results.

We use the function below to simulate this process. It takes three arguments:
1. `grid_size`: The size of the grid on which the ant is moving.
2. `n_iter`: The number of moves the ant makes.
3. `interval`: The interval at which the state of the grid is recorded.

In the function, two main grids are initialized. The first, `grid`, is a 2D grid representing the current state of the ant's world. The other, `grid_states`, is a 3D grid that will store the state of the grid every `interval` iterations. Why don't we record every iteration as before? To get an interesting observation from the ant, we will need to simulate over 10,000 iterations. Storing all of these iterations is computationally inefficient, and looking at each individual one won't be necessary.

We initialize our ant in the middle of the top-left corner of the grid and start it facing north. The code represents the direction as an integer (0 = north, 1 = east, 2 = south, 3 = west). In this way, adding one to the direction corresponds to a right turn and subtracting one corresponds to a left turn. We perform this operation *modulo 3*, meaning that adding 1 to 3 results in 0 while subtracting 1 from 0 results in 3. Think of the numbers as "wrapping around" the number 3.

The ant then moves based on the rule set defined above. It moves one node forward each iteration based on its current direction. If it moves off the grid, it loops back onto the grid on the opposite side. Again, we use the modulus operator here, this time modulo `grid_size`. It then recolours the node and changes direction as appropriate.

Finally, each `interval` iterations, the state of the grid is recorded in `grid_states`. At the end of the simulation, the function returns `grid_states`. We define the grid size, the number of iterations, and the interval at which the state of the grid is recorded and the use `visualize_ant_agent` to visualize the ant's walk. This function uses our `simulate_ant_agent` function to simulate the ant.

In [ ]:
## HIGHWAY BUILDING BY LANGTON'S ANT

def simulate_ant_agent(grid_size: int, n_iter: int, interval: int) -> np.ndarray:
    """Simulates an ant agent traversing a grid world with the following rules:

    - It moves one node forward every iteration.
    - If it lands on a white node, it turns it black and turns right.
    - If it lands on a black node, it turns it white and turns left.

    Args:
        grid_size: The size of the grid for the cellular automaton.
        n_iter: The number of iterations to simulate the cellular automaton.
        interval: The interval at which the record grid state is recorded.

    Returns:
        A 3D array representing the states of the grid world over time. The first dimension is time, and the second and
        third are the rows and columns of the grid. For efficiency, we only record the grid world every `interval`
        iterations.
    """
    # Initialize a 2D array for our grid world, with width and height `grid_size`.
    grid = np.zeros((grid_size, grid_size), dtype="int")

    # Initialize a 3D array for our grid world to record the state of the grid world every `interval` iterations.
    grid_states = np.zeros((n_iter // interval + 1, grid_size, grid_size), dtype="int")

    # This variable is the current index of the recorded grid worlds.
    # We start it at one since the first iteration is just the initial state.
    record_index = 1

    # 0 = north, 1 = east, 2 = south, 3 = west
    # To begin, we initialize the agent's direction to north. Incrementing by one turns the agent right. Decrementing by
    # one turns the agent left. This value should loop around, so incrementing by one past 3 results in 0. We'll do this
    # by using the modulus operator (%).
    agent_direction = 0

    # We position the agent in the middle of the top-left corner of the grid.
    agent_position_row = grid_size // 4
    agent_position_col = grid_size // 4
    grid[agent_position_row, agent_position_col] = 1
    grid_states[0, agent_position_row, agent_position_col] = 1

    # Now, we simulate the agent for `n_iter` steps.
    for step in range(1, n_iter + 1):
        # Move the agent forward by one node.
        # Case: the agent faces north, so decrement the row by 1.
        if agent_direction == 0:
            agent_position_row -= 1

        # Case: the agent faces east, so increment the column by 1.
        elif agent_direction == 1:
            agent_position_col += 1

        # Case: the agent faces south, so increment the row by 1.
        elif agent_direction == 2:
            agent_position_row += 1

        # Case: the agent faces west, so decrement the column by 1.
        elif agent_direction == 3:
            agent_position_col -= 1

        # Contain the agent within the grid.
        # Using the modulus operator (%), if the agent leaves the grid, it will just loop back on the opposite side.
        agent_position_row = agent_position_row % grid_size
        agent_position_col = agent_position_col % grid_size

        # Color the node the agent is on the opposite colour (0 = white, 1 = black) and update the agent's direction.
        if grid[agent_position_row, agent_position_col] == 0:
            # Color the node black.
            grid[agent_position_row, agent_position_col] = 1

            # Turn the agent right.
            agent_direction = (agent_direction + 1) % 4
        else:
            # Color the node black.
            grid[agent_position_row, agent_position_col] = 0

            # Turn the agent left.
            agent_direction = (agent_direction - 1) % 4

        # Record the grid state every `interval` iterations.
        if step % interval == 0:
            grid_states[record_index] = grid
            record_index += 1

    return grid_states


# Define the size of the grid.
grid_size = 300

# Define the number of iterations we want the agent to run.
n_iter = 51000

# Define the interval at which we record the grid state.
interval = 1000

# Use a custom plotting function to visualize the ant in its grid world.
visualize_ant_agent(grid_size, n_iter, interval)

The visualization shows that the ant's path seems random for the first few thousand steps and creates a messy mix of white and black nodes on the grid. But after about 10,000 steps, the ant starts behaving differently. It moves back and forth but also drifts diagonally across the grid, creating a neat, repeating pattern of white and black nodes. This pattern looks like a road or "highway" and is surprisingly orderly, given the ant's simple rules. This "highway building" always happens along 45-degree diagonals and would go on forever if the grid was infinitely big.

When we run simulations like this one, we usually do it on a grid that loops around, like a video game world where you walk off one edge and return on the other. So, when the ant keeps going, it eventually leaves the grid at the bottom-right corner and pops back in at the top-left corner, still moving in the same direction. When it does this, it'll run into the path it just made, which makes the ant go kind of crazy. For a while, it paints the grid randomly, as it did in the first 10,000 steps. But, after many steps, it starts making a highway again, but now it's going in a different direction, to the bottom-left. Over time, as the grid gets filled with random patches and highways crossing each other, it gets harder and harder for the ant to keep building highways. The whole grid will eventually look random if the simulation runs long enough.

This notebook and its source chapter in Charbonneau's book only scratch the surface of the world of cellular automata. The important takeaway is that simple rules can lead to seemingly complicated results. But does that make those results truly "complex"? Or do we call them "complex" just because they look intricate? This is a question that people studying complexity have been pondering for a long time.

Defining complexity is tough, and measuring it clearly and universally might be even tougher. There are many ways we can measure complexity. Charbonneau poses three questions to categorize complexity:
1. How hard is it to describe?
2. How hard is it to create?
3. How is it dynamically organized?

Think about a measure called *algorithmic complexity*, which refers to how long the shortest computer program would be that can make a particular result - like a pattern. It seems logical that more complex patterns would need longer programs. However, our study of 1D cellular automata showed this is only sometimes true. The four different cellular automata we looked at can be made with programs of the same length, but they aren't equally complex.

In studying cellular automata, we also encountered a common characteristic of complex systems: *emergence*. This term is used when a system has features or behaviours that can't be predicted just by looking at its parts. These new features or behaviours *emerge* from how the parts interact. The snowflakes and ant highways we saw are examples of this.

# References
* Charbonneau, P. (2017). *Natural complexity: A modeling handbook*. Princeton University Press.
* Petroff, M. A. (2021). *Accessible Color Sequences for Data Visualization* (arXiv:2107.02270). arXiv. http://arxiv.org/abs/2107.02270